In [1]:
from scipy import stats
import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

import statsmodels.api as sm 
import pylab as py 

# Se carga el dataset a partir del archivo CSV
weather_df  = pd.read_csv('dataset/weatherAUS.csv')

In [2]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

## Codificación de variable Location

Tal como se analizó previamente, la variable location es de tipo cualitativa y presenta 49 valores diferentes, lo cual no es tan elevado relativo al tamaño del dataset. Por lo tanto, una técnica de codificación que puede resultar adecuada en este caso es dummy encoding.

In [3]:
weather_df['Location'].unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

In [4]:
weather_df_codificado = pd.get_dummies(weather_df, columns=['Location'], dummy_na=True, drop_first=True)
display(weather_df_codificado.head(5))

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Location_Tuggeranong,Location_Uluru,Location_WaggaWagga,Location_Walpole,Location_Watsonia,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera,Location_nan
0,2008-12-01,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,False,False,False,False,False,False,False,False,False,False
1,2008-12-02,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,False,False,False,False,False,False,False,False,False,False
2,2008-12-03,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,False,False,False,False,False,False,False,False,False,False
3,2008-12-04,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,False,False,False,False,False,False,False,False,False,False
4,2008-12-05,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,False,False,False,False,False,False,False,False,False,False


In [5]:
weather_df_codificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 71 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Date                       145460 non-null  object 
 1   MinTemp                    143975 non-null  float64
 2   MaxTemp                    144199 non-null  float64
 3   Rainfall                   142199 non-null  float64
 4   Evaporation                82670 non-null   float64
 5   Sunshine                   75625 non-null   float64
 6   WindGustDir                135134 non-null  object 
 7   WindGustSpeed              135197 non-null  float64
 8   WindDir9am                 134894 non-null  object 
 9   WindDir3pm                 141232 non-null  object 
 10  WindSpeed9am               143693 non-null  float64
 11  WindSpeed3pm               142398 non-null  float64
 12  Humidity9am                142806 non-null  float64
 13  Humidity3pm                14

Con este método se crearon 48 columnas correspondientes a todos los valores posibles menos uno (en este caso Adelaide), ya que es posible inferirlo a partir de los valores de las restantes.

In [6]:
# La nueva columna Location_nan parece no tener sentido, ya que se había analizado previamente que no había valores nulos para Location
len(weather_df_codificado[weather_df_codificado['Location_nan'] == True])

0

Efectivamente, no hay valores verdaderos para "Location_nan", por lo tanto se elimina dicha columna.

In [7]:
del weather_df_codificado['Location_nan']

## Codificación de variables WindGustDir, WindDir9am y WindDir3pm

In [8]:
weather_df['WindGustDir'].unique()

array(['W', 'WNW', 'WSW', 'NE', 'NNW', 'N', 'NNE', 'SW', nan, 'ENE',
       'SSE', 'S', 'NW', 'SE', 'ESE', 'E', 'SSW'], dtype=object)

Dada la naturaleza circular de las direcciones del viento, no parece conveniente utilizar una técnica que establezca un orden lineal entre las mismas, dado que en cierto punto sucedería que dos direcciones similares terminen muy alejadas.

Por ejemplo, si codificamos partiendo desde N (Norte) en sentido horario quedaría:

N: 1, NNE: 2, NE: 3, ENE: 4, E: 5, ESE: 6, SE: 7, SSE: 8, S: 9, SSW: 10, SW: 11, WSW: 12, W: 13, WNW: 14, NW: 15, NNW: 16

Se presenta aquí el problema de que, por ejemplo, NNW (valor 16) y N (valor 1) están muy alejados pese a que en la realidad son direcciones cercanas.

Una alternativa es utilizar una codificación circular de la siguiente manera:

1) Asignar a cada punto cardinal un ángulo.
2) Crear dos nuevas columnas (eliminando la original) que indiquen el seno y el coseno de dicho ángulo.

De esta forma, este par de valores será similar para direcciones cercanas y más alejado para direcciones opuestas o ángulos rectos.

| Dirección | Ángulo (grados) | Ángulo (radianes) | Seno  | Coseno |
|-----------|-----------------|-------------------|-------|--------|
| N         | 0               | 0                 | 0.00  | 1.00   |
| NNE       | 22.5            | 0.392699081698724 | 0.38  | 0.92   |
| NE        | 45              | 0.785398163397448 | 0.71  | 0.71   |
| ENE       | 67.5            | 1.17809724509617  | 0.92  | 0.38   |
| E         | 90              | 1.5707963267949   | 1.00  | 0.00   |
| ESE       | 112.5           | 1.96349540849362  | 0.92  | -0.38  |
| SE        | 135             | 2.35619449019234  | 0.71  | -0.71  |
| SSE       | 157.5           | 2.74889357189107  | 0.38  | -0.92  |
| S         | 180             | 3.14159265358979  | 0.00  | -1.00  |
| SSW       | 202.5           | 3.53429173528852  | -0.38 | -0.92  |
| SW        | 225             | 3.92699081698724  | -0.71 | -0.71  |
| WSW       | 247             | 4.31096325242599  | -0.92 | -0.39  |
| W         | 270             | 4.71238898038469  | -1.00 | 0.00   |
| WNW       | 292.5           | 5.10508806208341  | -0.92 | 0.38   |
| NW        | 315             | 5.49778714378214  | -0.71 | 0.71   |
| NNW       | 337.5           | 5.89048622548086  | -0.38 | 0.92   |


In [9]:
direccion_to_angulo = {
    'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5, 'E': 90, 'ESE': 112.5,
    'SE': 135, 'SSE': 157.5, 'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
    'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5
}

def codificacion_wind_dir(df, columna, direccion_to_angulo):
    # Se obtienen los ángulos a partir del mapeo
    angulos = df[columna].map(direccion_to_angulo)

    # Se convierten los ángulos a radianes
    angulos_rad = np.deg2rad(angulos)

    # Se crean las nuevas columnas con el seno y el coseno
    df[f'{columna}_sin'] = np.sin(angulos_rad)
    df[f'{columna}_cos'] = np.cos(angulos_rad)

    # Se setea NaN en las nuevas columnas para aquellas direcciones que sean nulas.
    df.loc[df[columna].isna(), [f'{columna}_sin', f'{columna}_cos']] = np.nan

    # Se elimina la columna original
    del df[columna]
    #return df

codificacion_wind_dir(weather_df_codificado, 'WindGustDir', direccion_to_angulo)
codificacion_wind_dir(weather_df_codificado, 'WindDir9am', direccion_to_angulo)
codificacion_wind_dir(weather_df_codificado, 'WindDir3pm', direccion_to_angulo)

In [10]:
weather_df_codificado.head(5)

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,Location_Williamtown,Location_Witchcliffe,Location_Wollongong,Location_Woomera,WindGustDir_sin,WindGustDir_cos,WindDir9am_sin,WindDir9am_cos,WindDir3pm_sin,WindDir3pm_cos
0,2008-12-01,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,...,False,False,False,False,-1.000000,-1.836970e-16,-1.000000,-1.836970e-16,-0.923880,3.826834e-01
1,2008-12-02,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,...,False,False,False,False,-0.923880,3.826834e-01,-0.382683,9.238795e-01,-0.923880,-3.826834e-01
2,2008-12-03,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,...,False,False,False,False,-0.923880,-3.826834e-01,-1.000000,-1.836970e-16,-0.923880,-3.826834e-01
3,2008-12-04,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,...,False,False,False,False,0.707107,7.071068e-01,0.707107,-7.071068e-01,1.000000,6.123234e-17
4,2008-12-05,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,...,False,False,False,False,-1.000000,-1.836970e-16,0.923880,3.826834e-01,-0.707107,7.071068e-01
